## Logistic Regression - 8K Forms & Numerical Features

In [26]:
import time
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft_clean.pkl")
df.head()

,texts,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,VIX,label1,label2,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0,Mkt_Cap
0,Item 2.02\nResults of Operations and Financial...,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,86355.36
1,Item 2.02 Results of Operations and Financial ...,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,20.47,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,125102.08
2,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.07,neg,neg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,126812.46
3,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,139273.80
4,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.51,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,144649.28


In [6]:
def label1(row):
    if row["pct_change1"] > row["beta1"]*row["mkt_excess1"]:
        return "pos"
    else:
        return "neg"

def label2(row):
    if row["pct_change2"] > row["beta2"]*row["mkt_excess2"]:
        return "pos"
    else:
        return "neg"

def label3(row):
    if row["pct_change2"] > row["beta2"]*row["mkt_excess2"] + 0.005:
        return "positive"
    elif row["pct_change2"] < row["beta2"]*row["mkt_excess2"] - 0.005:
        return "negative"
    else:
        return "neutral"

In [7]:
df["label1"] = df.apply(label1, axis = 1)
df["label2"] = df.apply(label2, axis = 1)
df["label3"] = df.apply(label3, axis = 1)

In [8]:
# get column names of categorical features
item_list = [i for i in df.columns if not i.find("item")]
ind_list = [i for i in df.columns if not i.find("ind")]
shrcd_list = [i for i in df.columns if not i.find("SHRCD")]
exc_list = [i for i in df.columns if not i.find("EXCHCD")]

In [9]:
# combine all feature column names
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

In [10]:
# Separate into features, documents and label
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

X = df[cols]

docs = df['texts']

y = df['label2']

#### Process texts

In [11]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [12]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
        
    return text

In [13]:
# apply clean function to documents
docs = df.texts.apply(clean_text)
docs.tail()

21697    [item, 2, 02, results, operations, financial, ...
21698    [item, 5, 07, submission, matters, vote, secur...
21699    [item, 5, 02, departure, directors, certain, o...
21700    [item, 2, 02, results, operations, financial, ...
21701    [item, 5, 07, submission, matters, vote, secur...
Name: texts, dtype: object

In [14]:
# lemmatize documents
def lemmatized_words(text):
    lemm = nltk.stem.WordNetLemmatizer()
    return list(map(lambda word: list(map(lemm.lemmatize, word)), docs))
    
lemmatized = lemmatized_words(docs)

In [25]:
# Split into train and test data
X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y, lemmatized,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)

In [27]:
# standardize volatility index and market cap by mean and standard deviation (standardize train and test sets separately)
x_scaler = StandardScaler()
X_train["VIX"] = x_scaler.fit_transform(np.array(X_train["VIX"]).reshape(-1,1))
X_test["VIX"] = x_scaler.transform(np.array(X_test["VIX"]).reshape(-1,1))
X_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(X_train["Mkt_Cap"]).reshape(-1,1))
X_test["Mkt_Cap"] = x_scaler.transform(np.array(X_test["Mkt_Cap"]).reshape(-1,1))

In [28]:
# define BoW transformer, set max_features to 4000 to balance the dimensions with numerical features
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, max_features=4000, ngram_range=[1,3], lowercase=False)

In [30]:
#transform documents
X_tr_bow = bow_transform.fit_transform(docs_train)
X_te_bow = bow_transform.transform(docs_test)

In [31]:
X_tr_bow.shape

(17361, 4000)

In [32]:
#concatenate BoW features with numerical features (convert Bow features to dense array first)
X_tr_bow_ft = np.concatenate((X_tr_bow.toarray(), np.array(X_train)), axis = 1)
X_te_bow_ft = np.concatenate((X_te_bow.toarray(), np.array(X_test)), axis = 1)

In [34]:
# define Tf-idf transformer, set max_features to 4000 to balance the dimensions with numerical features
tfidf_transform = TfidfVectorizer(tokenizer=lambda doc: doc, max_features=4000, ngram_range=[1,3], lowercase=False)

In [35]:
#transform documents
X_tr_tfidf = tfidf_transform.fit_transform(docs_train)
X_te_tfidf = tfidf_transform.transform(docs_test)

In [36]:
#concatenate Tf-idf features with numerical features (convert Tf-idf features to dense array first)
X_tr_tfidf_ft = np.concatenate((X_tr_tfidf.toarray(), np.array(X_train)), axis = 1)
X_te_tfidf_ft = np.concatenate((X_te_tfidf.toarray(), np.array(X_test)), axis = 1)

In [33]:
# define logistic regression classifier
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

In [37]:
# fit the model to (BoW + numerical) and (Tf-idf + numerical) features
model_bow = simple_logistic_classify(X_tr_bow_ft, y_train, X_te_bow_ft, y_test, 'bow')
model_tfidf = simple_logistic_classify(X_tr_tfidf_ft, y_train, X_te_tfidf_ft, y_test, 'tf-idf')

Test Score with bow features 0.5807417645703755
Test Score with tf-idf features 0.6056208246947707


In [38]:
# Hyperparameter tuning

# grid search parameters
param_grid_ = [{'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2], 'penalty':["l1", "l2", "elasticnet"]}]

#create grid search with 5-fold cross validation
bow_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5,
                                   param_grid=param_grid_)

In [39]:
bow_search.fit(X_tr_bow_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [40]:
bow_search.best_score_

0.5969705403501137

In [41]:
tfidf_search.fit(X_tr_tfidf_ft, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [42]:
tfidf_search.best_score_

0.6077993534056667

In [43]:
# get best models' predictions
bow_pred = bow_search.predict(X_te_bow_ft)
tfidf_pred = tfidf_search.predict(X_te_tfidf_ft)

In [44]:
print(classification_report(y_test, bow_pred))

              precision    recall  f1-score   support

         neg       0.57      0.34      0.43      1935
         pos       0.60      0.79      0.68      2406

    accuracy                           0.59      4341
   macro avg       0.59      0.57      0.56      4341
weighted avg       0.59      0.59      0.57      4341



In [45]:
print(classification_report(y_test, tfidf_pred))

              precision    recall  f1-score   support

         neg       0.58      0.44      0.50      1935
         pos       0.62      0.74      0.67      2406

    accuracy                           0.61      4341
   macro avg       0.60      0.59      0.59      4341
weighted avg       0.60      0.61      0.60      4341



### Multi-Class Logistic Regression

In [46]:
y = df['label3']

We have to repeat the process, because classes (split&stratification) changed.

In [47]:
# Split into train and test data
X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y, lemmatized,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)

In [48]:
X_train["VIX"] = x_scaler.fit_transform(np.array(X_train["VIX"]).reshape(-1,1))
X_test["VIX"] = x_scaler.transform(np.array(X_test["VIX"]).reshape(-1,1))
X_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(X_train["Mkt_Cap"]).reshape(-1,1))
X_test["Mkt_Cap"] = x_scaler.transform(np.array(X_test["Mkt_Cap"]).reshape(-1,1))

In [51]:
#transform documents
X_tr_tfidf = tfidf_transform.fit_transform(docs_train)
X_te_tfidf = tfidf_transform.transform(docs_test)
X_tr_bow = bow_transform.fit_transform(docs_train)
X_te_bow = bow_transform.transform(docs_test)

In [50]:
#concatenate text vectors with numerical features (convert sparse matrix to dense matrix first)
X_tr_tfidf_ft = np.concatenate((X_tr_tfidf.toarray(), np.array(X_train)), axis = 1)
X_te_tfidf_ft = np.concatenate((X_te_tfidf.toarray(), np.array(X_test)), axis = 1)
X_tr_bow_ft = np.concatenate((X_tr_bow.toarray(), np.array(X_train)), axis = 1)
X_te_bow_ft = np.concatenate((X_te_bow.toarray(), np.array(X_test)), axis = 1)

In [52]:
# define MULTINOMIAL logistic regression classifier
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C, multi_class="multinomial").fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

In [53]:
# fit the model
model_bow = simple_logistic_classify(X_tr_bow_ft, y_train, X_te_bow_ft, y_test, 'bow')
model_tfidf = simple_logistic_classify(X_tr_tfidf_ft, y_train, X_te_tfidf_ft, y_test, 'tf-idf')

Test Score with bow features 0.36788758350610457
Test Score with tf-idf features 0.47500575904169545


In [60]:
tfidf_pred = model_tfidf.predict(X_te_tfidf_ft)

In [59]:
print(classification_report(y_test, tfidf_pred))

              precision    recall  f1-score   support

    negative       0.47      0.40      0.43      1289
     neutral       0.46      0.41      0.43      1364
    positive       0.48      0.59      0.53      1688

    accuracy                           0.48      4341
   macro avg       0.47      0.46      0.47      4341
weighted avg       0.47      0.48      0.47      4341

